In [1]:
import pandas as pd
import time
import re
import datetime

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.parse import urljoin
from konlpy.tag import Okt, Hannanum
from tqdm.notebook import tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [3]:
driver = webdriver.Chrome('/HDD/workspace/Analysis-of-Corporate-Bankruptcy/chromedriver')
driver.implicitly_wait(3)
driver.get('https://search.naver.com/search.naver?where=news')

In [4]:
name_list = '에이앤티앤'

In [5]:
driver.find_element_by_name('query').send_keys(name_list)
driver.find_element_by_class_name('bt_search').click()

In [6]:
title = []
news_date = []
word_data = []

In [7]:
date_re = re.compile(r'\d\d\d\d.\d\d.\d\d.')

In [8]:
while True:
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    title_list = soup.find_all(class_='_sp_each_title')
    date = soup.find_all(class_='txt_inline')
    word = soup.find_all(class_='type01')[0].find_all('dl')

    for i in title_list:
        title.append(i.get_text())

    for dates in date:
        try:
            match = date_re.search(dates.get_text())
            news_date.append(match.group())
        except:
            date_re2 = re.compile(r'\d')
            match = date_re2.search(dates.get_text())
            m = int(match.group())
            year = str(datetime.datetime.today().year)
            month = str(datetime.datetime.today().month)
            day = str(datetime.datetime.today().day-m)
            result = year + '.' + month + '.' + day + '.'
            news_date.append(result)
    
    for words in word:
        word_data.append(words.find_all('dd')[1].get_text())
    try:
        if soup.find_all(class_='next')[0].get_text() != '다음페이지':
            break
        else: 
            driver.find_element_by_class_name('next').click()
            driver.implicitly_wait(1)
    except:
        if soup.find(class_='next') != '다음페이지':
            break

In [9]:
print(len(title), len(news_date), len(word_data))

812 812 812


In [10]:
df = pd.DataFrame([title, news_date, word_data])

In [11]:
df = df.T

In [12]:
df

,0,1,2
0,"[IB토마토](Today 공시)두산건설, 2분기 885억원 당기순손실",2020.08.14.,"에이앤티앤(050320), 반기보고서 감사의견 '의견거절' 에이앤티앤은 올해 반기보..."
1,"에이앤티앤, '아래스'로 상호 변경",2020.07.27.,에이앤티앤(050320)은 상호를 아래스 주식회사로 변경한다고 27일 공시했다. 합...
2,"에이앤티앤, 특별관계자 지분변동",2020.05.28.,에이앤티앤은 주식회사 티이알 및 특별관계자의 지분율이 0%에서 13.59%로 변동했...
3,"증선위, KT&G 분식회계 '고의성 없다' 최종 결론…검찰고발 피해",2020.07.15.,"상장사 에이앤티앤에 대해서는 검찰 고발과 임원 면직 권고, 과징금 부과(부과액은 금..."
4,에이앤티앤 “회계처리 위반으로 과징금·檢고발 조치받아”,2020.07.23.,에이앤티앤(050320)은 회계처리기준 위반 행위로 인해 지난 15일 증권선물위원회...
...,...,...,...
807,"[표](29일 장마감 기준) 코스피 코스닥 주식 거래 정지 종목 현황..자비스,스...",2019.10.29.,"▲ 종목명 : 에이앤티앤, 에이앤티앤 거래정지일 : 2019.05.31, 에이앤티앤..."
808,"거래소 ""코스피·코스닥 상장사 중 41개사 관리종목 지정""",2019.08.16.,"라이트론, 에이앤티앤, 썬텍, 테라셈 등은 반기검토(감사)의견 비적정, 자본잠식률 ..."
809,[표](28일 장마감 기준) 코스피 코스닥 주식 거래 정지 종목 현황...에이치엘...,2019.10.28.,"▲ 종목명 : 에이앤티앤, 에이앤티앤 거래정지일 : 2019.05.31, 에이앤티앤..."
810,[표](5일 장마감 기준) 코스피 코스닥 주식 거래 정지 종목 현황..더블유에프...,2019.12.05.,"▲ 종목명 : 에이앤티앤, 에이앤티앤 거래정지일 : 2019.05.31, 에이앤티앤..."


In [13]:
df[2] = df[2].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]"," ")

In [14]:
df[2]

0      에이앤티앤          반기보고서 감사의견  의견거절  에이앤티앤은 올해 반기보...
1      에이앤티앤        은 상호를 아래스 주식회사로 변경한다고   일 공시했다  합...
2      에이앤티앤은 주식회사 티이알 및 특별관계자의 지분율이   에서       로 변동했...
3      상장사 에이앤티앤에 대해서는 검찰 고발과 임원 면직 권고  과징금 부과 부과액은 금...
4      에이앤티앤        은 회계처리기준 위반 행위로 인해 지난   일 증권선물위원회...
                             ...                        
807      종목명   에이앤티앤  에이앤티앤 거래정지일               에이앤티앤...
808    라이트론  에이앤티앤  썬텍  테라셈 등은 반기검토 감사 의견 비적정  자본잠식률 ...
809      종목명   에이앤티앤  에이앤티앤 거래정지일               에이앤티앤...
810      종목명   에이앤티앤  에이앤티앤 거래정지일               에이앤티앤...
811    경남제약  셀바스    에이앤티앤  인터불스     이나그레이트 등   개사는 신규...
Name: 2, Length: 812, dtype: object

In [21]:
stopwords = ['을', '로', '의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다''의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

hannanum = Hannanum()

In [22]:
contents = []

for sentence in tqdm(df[2]):
    temp_x = []
    temp_x = hannanum.nouns(sentence)
    temp_x = [word for word in temp_x if not word in stopwords]
    contents.append(temp_x)

In [24]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(contents)
sequences = tokenizer.texts_to_sequences(contents)

In [25]:
print(len(sentence))

122


In [26]:
word_to_index = tokenizer.word_index
index_to_word = {}
for key, value in word_to_index.items():
    index_to_word[value] = key
len(word_to_index)

2021

In [28]:
for i in range(1,10):
    print(index_to_word[i])

에이앤티앤
일
코오롱티슈진
거래정지
종목명
지정
사유
공시
등
